# Benchmarking tf-shell

In [1]:
import tf_shell
import tensorflow as tf
import timeit

context = tf_shell.create_context64(
    log_n=10,
    main_moduli=[8556589057, 8388812801],
    plaintext_modulus=40961,
    scaling_factor=3,
    seed="test_seed",
)

secret_key = tf_shell.create_key64(context)
rotation_key = tf_shell.create_rotation_key64(context, secret_key)

a = tf.random.uniform([context.num_slots, 55555], dtype=tf.float32, maxval=10)
b = tf.random.uniform([55555, 333], dtype=tf.float32, maxval=10)
c = tf.random.uniform([2, context.num_slots], dtype=tf.float32, maxval=10)
d = tf.random.uniform([context.num_slots, 4444], dtype=tf.float32, maxval=10)

enc_a = tf_shell.to_encrypted(a, secret_key, context)

2024-11-10 01:15:42.366708: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-10 01:15:42.367179: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-10 01:15:42.369814: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-10 01:15:42.376276: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731201342.387674  117259 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731201342.39

INFO: Generating key
INFO: Generating rotation key
INFO: Generating rotation key


In [2]:
def to_pt():
    return tf_shell.to_shell_plaintext(a, context)

time = min(timeit.Timer(to_pt).repeat(repeat=3, number=1))
print(time)

0.5067476989970601


In [3]:
def enc():
    return tf_shell.to_encrypted(d, secret_key, context)

time = min(timeit.Timer(enc).repeat(repeat=3, number=1))
print(time)

0.1824721849989146


In [4]:
def dec():
    return tf_shell.to_tensorflow(enc_a, secret_key)

time = min(timeit.Timer(dec).repeat(repeat=3, number=1))
print(time)

0.5525883640002576


In [5]:
def ct_ct_add():
    return enc_a + enc_a

time = min(timeit.Timer(ct_ct_add).repeat(repeat=3, number=1))
print(time)

0.18785935000050813


In [6]:
def ct_ct_sub():
    return enc_a - enc_a

time = min(timeit.Timer(ct_ct_sub).repeat(repeat=3, number=1))
print(time)

0.19354859399754787


In [7]:
def ct_ct_mul():
    return enc_a * 4

time = min(timeit.Timer(ct_ct_mul).repeat(repeat=3, number=1))
print(time)

0.17636438800036558


In [8]:
def ct_pt_add():
    return enc_a + a

time = min(timeit.Timer(ct_pt_add).repeat(repeat=3, number=1))
print(time)

0.7591958359989803


In [9]:
def ct_pt_mul():
    return enc_a * a

time = min(timeit.Timer(ct_pt_mul).repeat(repeat=3, number=1))
print(time)

0.7197426070015354


In [10]:
def ct_pt_matmul():
    return tf_shell.matmul(enc_a, b)

time = min(timeit.Timer(ct_pt_matmul).repeat(repeat=3, number=1))
print(time)

27.007230432998767


In [11]:
def pt_ct_matmul():
    return tf_shell.matmul(c, enc_a, rotation_key)

time = min(timeit.Timer(pt_ct_matmul).repeat(repeat=3, number=1))
print(time)

370.09557524000047


In [12]:
def ct_roll():
    return tf_shell.roll(enc_a, 2, rotation_key)

time = min(timeit.Timer(ct_roll).repeat(repeat=3, number=1))
print(time)

5.250123850997625
